# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
from sqlalchemy import create_engine
import matplotlib
import seaborn as sns
import pandas as pd

In [2]:
# load messages dataset
messages = pd.read_csv('messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
# load categories dataset
categories = pd.read_csv('categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [4]:
# merge datasets
df = pd.merge(messages, categories, on='id')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [5]:
# create a dataframe of the 36 individual category columns
categories = df.categories.str.split(";", expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [6]:
# select the first row of the categories dataframe
row = categories.iloc[0]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = [item[:-2] for item in row]
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [7]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [8]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].apply(lambda x: x[-1])
    
    # convert column from string to numeric
    categories[column] = categories[column].astype(int)
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [9]:
# drop the original categories column from `df`
df.drop('categories', axis=1, inplace=True)

df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [10]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis=1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [11]:
# check number of duplicates
df.duplicated().sum()

170

In [12]:
# drop duplicates
df.drop_duplicates(inplace=True)

In [13]:
# check number of duplicates
df.duplicated().sum()

0

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [14]:
engine = create_engine('sqlite:///categorized_messages.db')
df.to_sql('categorized_messages', engine, index=False, if_exists='replace')

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

### Further investigation and refinement

Wrap-up: What are the columns?

In [15]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [16]:
df[df.isna().any(axis=1)].shape

(16046, 40)

In [17]:
df[df[['message'] + category_colnames].isna().any(axis=1)].shape

(0, 40)

There are missing entries, however they don't appear in the relevant columns.  
***-> Drop rows with missing values in relevant columns.***

In [18]:
(df.isna() == False).sum()

id                        26216
message                   26216
original                  10170
genre                     26216
related                   26216
request                   26216
offer                     26216
aid_related               26216
medical_help              26216
medical_products          26216
search_and_rescue         26216
security                  26216
military                  26216
child_alone               26216
water                     26216
food                      26216
shelter                   26216
clothing                  26216
money                     26216
missing_people            26216
refugees                  26216
death                     26216
other_aid                 26216
infrastructure_related    26216
transport                 26216
buildings                 26216
electricity               26216
tools                     26216
hospitals                 26216
shops                     26216
aid_centers               26216
other_in

As later only a categories are inferred from messages (in english), 'id', 'original' and 'genre' can be dropped.

In [19]:
df.drop(['id', 'original', 'genre'], axis=1, inplace=True)

In [20]:
[df[category_colnames][category].unique() for category in category_colnames]

[array([1, 0, 2]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1])]

'child_alone' doesn't have varying entries (only 0).  
***-> Category columns with no varying entries could be dropped. But unsure due to future behaviour.***

In [21]:
df.related.value_counts()

1    19906
0     6122
2      188
Name: related, dtype: int64

'related' only category having three values 0, 1, 2, with 2 only occurring 188 times.

In [22]:
pd.concat([df[(df.related == 0) | (df.related == 2)][category_colnames].min(), 
           df[(df.related == 0) | (df.related == 2)][category_colnames].max()], 
          axis=1)

,0,1
related,0,2
request,0,0
offer,0,0
aid_related,0,0
medical_help,0,0
medical_products,0,0
search_and_rescue,0,0
security,0,0
military,0,0
child_alone,0,0


'related' with 2 seems to behave like 'related' with 0 having all other categories 0. So, this is probably and error. However, as we can't decide on future behaviour:  
***-> Only accept 0 or 1 as entries in categories***

In [23]:
df.drop(df[(df[category_colnames] == 2).any(axis=1) == True].index, inplace=True)

In [24]:
df[category_colnames].sum()

related                   19906
request                    4474
offer                       118
aid_related               10860
medical_help               2084
medical_products           1313
search_and_rescue           724
security                    471
military                    860
child_alone                   0
water                      1672
food                       2923
shelter                    2314
clothing                    405
money                       604
missing_people              298
refugees                    875
death                      1194
other_aid                  3446
infrastructure_related     1705
transport                  1201
buildings                  1333
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1151
weather_related            7297
floods                     2155
storm                      2443
fire    

In [25]:
df[df.related == 0][category_colnames].sum()

related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

'related' seems to be meta-category (*Is the message disaster related?*, see [figure8](https://www.figure-eight.com/dataset/combined-disaster-response-data/))

In [26]:
for category in category_colnames:
    try:
        min_other_cats_if_1 = df[category_colnames].groupby(category).min().T[1]
        print("{:<20}".format(category),
              'being 1 is always related to', 
              min_other_cats_if_1[min_other_cats_if_1 == 1].index.values, 
              'being 1')
    except:
        print(category, "doesn't have 1s.")

related              being 1 is always related to [] being 1
request              being 1 is always related to ['related'] being 1
offer                being 1 is always related to ['related'] being 1
aid_related          being 1 is always related to ['related'] being 1
medical_help         being 1 is always related to ['related' 'aid_related'] being 1
medical_products     being 1 is always related to ['related' 'aid_related'] being 1
search_and_rescue    being 1 is always related to ['related' 'aid_related'] being 1
security             being 1 is always related to ['related' 'aid_related'] being 1
military             being 1 is always related to ['related' 'aid_related'] being 1
child_alone doesn't have 1s.
water                being 1 is always related to ['related' 'aid_related'] being 1
food                 being 1 is always related to ['related' 'aid_related'] being 1
shelter              being 1 is always related to ['related' 'aid_related'] being 1
clothing             being 1

In [27]:
df[['request', 'offer']].drop_duplicates(keep='last')

,request,offer
26284,0,1
26377,1,0
26385,0,0


'request' and 'offer' seem to be exclusive choices. However, not sure about future behaviour.

In [28]:
(pd.concat([
    (df[df[cat_col] == 1][category_colnames[3:-1]].sum() / df[df[cat_col] == 1].shape[0]) 
    for cat_col in category_colnames[3:-1]], 
    axis=1)
 .rename(columns={key: value for (key, value) in enumerate(category_colnames[3:-1])})
      .round(2)).T

,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,...,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather
aid_related,1.00,0.19,0.12,0.07,0.04,0.08,0.0,0.15,0.27,0.21,...,0.01,0.03,0.08,0.39,0.14,0.11,0.02,0.12,0.03,0.09
medical_help,1.00,1.00,0.30,0.09,0.05,0.04,0.0,0.18,0.21,0.17,...,0.01,0.03,0.07,0.43,0.26,0.10,0.02,0.11,0.04,0.08
medical_products,1.00,0.48,1.00,0.09,0.05,0.03,0.0,0.33,0.37,0.29,...,0.02,0.05,0.08,0.37,0.19,0.09,0.03,0.10,0.04,0.08
search_and_rescue,1.00,0.25,0.15,1.00,0.10,0.08,0.0,0.10,0.13,0.15,...,0.01,0.04,0.12,0.66,0.32,0.16,0.04,0.21,0.07,0.11
security,1.00,0.20,0.15,0.15,1.00,0.24,0.0,0.06,0.12,0.13,...,0.03,0.04,0.09,0.54,0.30,0.10,0.06,0.14,0.06,0.07
military,1.00,0.09,0.04,0.07,0.13,1.00,0.0,0.04,0.06,0.05,...,0.01,0.03,0.08,0.24,0.08,0.04,0.03,0.06,0.03,0.06
child_alone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
water,1.00,0.22,0.26,0.04,0.02,0.02,0.0,1.00,0.54,0.31,...,0.02,0.04,0.09,0.28,0.10,0.09,0.02,0.07,0.02,0.09
food,1.00,0.15,0.17,0.03,0.02,0.02,0.0,0.31,1.00,0.27,...,0.01,0.02,0.06,0.26,0.09,0.09,0.01,0.05,0.02,0.08
shelter,1.00,0.15,0.17,0.05,0.03,0.02,0.0,0.23,0.34,1.00,...,0.02,0.07,0.11,0.41,0.14,0.14,0.02,0.11,0.05,0.09


In [29]:
related = ['related']

request_offer = ['request', 'offer']

aid_related_top = ['aid_related']
aid_related_sub = [
    'medical_help', 
    'medical_products', 
    'search_and_rescue', 
    'security',
    'military',
    'child_alone',
    'water',
    'food',
    'shelter',
    'clothing',
    'money',
    'missing_people',
    'refugees',
    'death',
    'other_aid'
]
aid_related = aid_related_top + aid_related_sub

infrastructure_related_top = ['infrastructure_related']
infrastructure_related_sub = [
    'hospitals', 
    'shops', 
    'aid_centers', 
    'other_infrastructure'
]
infrastructure_related = infrastructure_related_top + infrastructure_related_sub

ambigous_related_top = ['ambigous_related']
ambigous_related_sub = [
    'transport',
    'buildings',
    'electricity',
    'tools']
ambigous_related = ambigous_related_top + ambigous_related_sub

weather_related_top = ['weather_related']
weather_related_sub = [
    'floods', 
    'storm', 
    'fire', 
    'earthquake', 
    'cold', 
    'other_weather'
]
weather_related = weather_related_top + weather_related_sub

tops = aid_related_top + infrastructure_related_top + weather_related_top
subs = aid_related_sub + ambigous_related_sub + infrastructure_related_sub+ weather_related_sub

direct_report = ['direct_report']

category_colnames == related + request_offer + aid_related + infrastructure_related_top + ambigous_related_sub + infrastructure_related_sub + weather_related + direct_report

True

In [30]:
def coverage(cats_to_cover, cover_by_cats):
    coverage = (
        pd.concat([(df[df[cat_col] == 1][category_colnames].sum() / df[df[cat_col] == 1].shape[0]) 
                   for cat_col in category_colnames], 
                  axis=1)
        .rename(columns={key: value for (key, value) in enumerate(category_colnames)})
        .round(2).T
        .loc[cats_to_cover, cover_by_cats])
    return coverage

In [31]:
coverage(subs, related + tops)

,related,aid_related,infrastructure_related,weather_related
medical_help,1.0,1.00,0.16,0.43
medical_products,1.0,1.00,0.17,0.37
search_and_rescue,1.0,1.00,0.20,0.66
security,1.0,1.00,0.18,0.54
military,1.0,1.00,0.14,0.24
child_alone,NaN,NaN,NaN,NaN
water,1.0,1.00,0.14,0.28
food,1.0,1.00,0.09,0.26
shelter,1.0,1.00,0.18,0.41
clothing,1.0,1.00,0.12,0.32


In [32]:
coverage(related + tops, subs).T

,related,aid_related,infrastructure_related,weather_related
medical_help,0.10,0.19,0.19,0.12
medical_products,0.07,0.12,0.13,0.07
search_and_rescue,0.04,0.07,0.09,0.07
security,0.02,0.04,0.05,0.04
military,0.04,0.08,0.07,0.03
child_alone,0.00,0.00,0.00,0.00
water,0.08,0.15,0.14,0.06
food,0.15,0.27,0.16,0.10
shelter,0.12,0.21,0.25,0.13
clothing,0.02,0.04,0.03,0.02


In [33]:
coverage(ambigous_related_sub, request_offer + direct_report)

,request,offer,direct_report
transport,0.21,0.0,0.26
buildings,0.27,0.0,0.32
electricity,0.20,0.0,0.33
tools,0.26,0.0,0.28


In [34]:
coverage(subs, request_offer + direct_report)

,request,offer,direct_report
medical_help,0.34,0.01,0.33
medical_products,0.35,0.02,0.30
search_and_rescue,0.33,0.00,0.33
security,0.33,0.00,0.34
military,0.08,0.00,0.11
child_alone,NaN,NaN,NaN
water,0.52,0.01,0.49
food,0.54,0.02,0.53
shelter,0.51,0.01,0.49
clothing,0.42,0.03,0.40


In [35]:
coverage(request_offer + direct_report, subs).T

,request,offer,direct_report
medical_help,0.16,0.10,0.13
medical_products,0.10,0.18,0.08
search_and_rescue,0.05,0.03,0.05
security,0.03,0.00,0.03
military,0.02,0.03,0.02
child_alone,0.00,0.00,0.00
water,0.19,0.08,0.16
food,0.36,0.41,0.30
shelter,0.26,0.11,0.22
clothing,0.04,0.12,0.03


In [36]:
df[df.aid_related == 1][aid_related_sub].max(axis=1).min()

1

In [37]:
df[df.infrastructure_related == 1][infrastructure_related_sub + ambigous_related_sub].max(axis=1).min()

1

In [38]:
df[df.weather_related == 1][weather_related_sub].max(axis=1).min()

1

- Although every category column being 1 implied 'related' being 1, it's not the other way around. To the contrary, there are more than 5,000 rows only having 'related' being 1.
- On the other hand, 'weather_related', 'infrastructure_related' and 'aid_related' seem to be kind of 'top' categories in a sense that one of their 'subcategories' being 1 implies the top-category being one (and other way around).
- 'request' and 'offer' as well as 'direct_report' are a category of their own

In [39]:
df[(df.related == 1) & ((df.request == 1) | (df.offer == 1))][request_offer + tops + ambigous_related_sub + direct_report].sum(axis=1).min() - 1

0

In [40]:
df[(df.related == 1) & (df.direct_report == 1)][request_offer + tops + ambigous_related_sub + direct_report].sum(axis=1).min() - 1

0

Neither 'request' or 'offer', nor 'direct_report' being 1 imply any other category being 1.

In [41]:
for message in df[df.direct_report == 1]['message'].sample(10):
    print(message)

WHERE CAN WE HAVE WATER FOR FREE? 
We are part of O. J. D. S. C. N ) KI SE OGanizasyon JEN POU DEVLOPMAN SOSYAL AK KILTIREL NAN NAZON ( Organization for National Social and Cultural Development ). Our base is at Pierre Baptiste. Please help us, we have no water.
what about the aid am i going to find it
PLEASE HELP ME GOOD LORD. WE COMMIT TO YOU OUR BROTHERS AND SISTERS.
Need help, we need tents, water, food, medicine please send the message
The country has also been experiencing extreme temperatures for the past few weeks , with the highest temperature recorded during the heat wave at 52 degrees Celsius (125.6 Fahrenheit).
Was that an earthquake or am i imagining things now?
WE need electricity at Leogane because if we have it we can amuse(distract) and every body can enjoy.Please,think about us. 
can u please help me I live in Carfour Lamantin 52 rue demokr
we haven't received anything in terms of aid. for example we nee food, medicine, doctors, and tents to sleep in. to be okay in ?S

In [42]:
for message in df[(df.direct_report == 0) & (df.related == 1)]['message'].sample(10):
    print(message)

The floods also destroyed irrigation networks and five km of roads in Padang and inundated thousands of hectares of paddy fields in Pesisir Selatan, Padang, Padang Pariaman and Agam.
Oxfam is working on supplying clean water and sanitation to nearly 80,400 people affected by the recent flooding.
Westnet we're in avenue Muller neighborhood and in Road St Alexandre.
Hello, I would like to have importain on what has happening. Thanks You
As people began to make a slow transition from tents and big public buildings to other temporary shelters, INFORM worked to ensure that these new shelters provided access to electricity, water, and sewage and garbage disposal, and that children could start returning to school.
Under its influence, some moisture penetration in upper parts of the country may start in the next 36-48 hours and can cause rains in the areas.
Three kilometers south, in Hikkaduwa, the wave height was just two to three meters, traveled only 50 meters inland and caused no deaths.
M

In [43]:
for message in df[(df.infrastructure_related == 1) & (df.related == 1)]['message'].sample(10):
    print(message)

The emergency operations centre (Control Room) at the National Headquarters is functioning round the clock to monitor the flood situation and coordinate rescue, relief and resource mobilization efforts.
Carefour Feiulles zone Savane Pistache. we haven't received anything please send this message. ..
GILGIT: Continuous rains triggering flashfloods and landslides have damaged 42 hydel power stations across the Gilgit-Baltistan region, disconnecting power supply to 90 per cent of the area, and damaging road infrastructure.
WE FOUND A LOT OF PEOPLE IN ( KOMIN MONBIN KWOCHI ) THOSE WHO ARE VICTIMS SOME LOST THEIR LIVES SOME LOST THEIR HOMES. ALL THIS PEOPLE LIVES IN ( MONBIN KWOCHI )
We've found some fifty houses which suffered mamages with 6 totally down a not enough food to feed people who came did not give us and not enough tools and seeds to plant, thanks a lot
Medair is working closely with established partners to identify remote and unreached villages in Sindhupalchowk, a mountainous 

In [44]:
cat_cols =[col for col in df.columns if col!='message']

cats = df[cat_cols].copy()

In [73]:
cats_by_cats_being_1 = pd.concat([cats[cats[col] == 1].mean() for col in cat_cols], axis=1).fillna(0)
#cats_by_cats_being_1.columns = cat_cols

cm = sns.light_palette("green", as_cmap=True)

#pd.set_option('colheader_justify', 1)
cats_by_cats_being_1.style.background_gradient(cmap=cm).format("{:.0%}").set_properties(**{'width': '10px'})
#pd.set_option('max_colwidth')